In [2]:
import pandas as pd

In [4]:
member = pd.read_csv("data/members.csv")
song_one = pd.read_csv("data/songs.csv")
song_two = pd.read_csv("data/song_extra_info.csv")
event = pd.read_csv("data/train.csv")

In [7]:
print(member.shape)
print(song_one.shape)
print(song_two.shape)
print(event.shape)

(34403, 7)
(2296320, 7)
(2295971, 3)
(7377418, 6)


In [8]:
print(abs(song_one.shape[0] - song_two.shape[0]))

349


Note song_one contains 349 more songs than song_two. 

In [16]:
event.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target'],
      dtype='object')

In [13]:
print(event['source_type'].unique())
print(event['source_screen_name'].unique())

['online-playlist' 'local-playlist' 'local-library' 'top-hits-for-artist'
 'album' nan 'song-based-playlist' 'radio' 'song' 'listen-with' 'artist'
 'topic-article-playlist' 'my-daily-playlist']
['Explore' 'Local playlist more' nan 'My library' 'Online playlist more'
 'Album more' 'Discover Feature' 'Unknown' 'Discover Chart' 'Radio'
 'Artist more' 'Search' 'Others profile more' 'Search Trends'
 'Discover Genre' 'My library_Search' 'Search Home' 'Discover New'
 'Self profile more' 'Concert' 'Payment']


Note there are NaNs in both source type and source screen name

In [19]:
print(event['source_type'].isna().sum(axis=0))
print(event['source_screen_name'].isna().sum(axis=0))

21539
414804


21539 out of 7377418 rows has NaN in source type
414804 out of "" in source_screen_name

In [17]:
print(event['msno'].nunique())
print(event['song_id'].nunique())

30755
359966


Taking a look at song_one

In [25]:
song_one.columns

Index(['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language'],
      dtype='object')

In [24]:
print(song_one['genre_ids'].unique())
print(song_one['genre_ids'].nunique())

['465' '444' '726' ... '786|958|2022' '1259|2107' '1138|2022']
1045


Looks like there are at most 1045 genre ids. Some of them are joint of multiple genre.
<b><span style="color:red">And kkbox didnt release legend to indicate what each of the genre corresponds to.</span><b>

In [26]:
song_one['artist_name'].unique()

array(['張信哲 (Jeff Chang)', 'BLACKPINK', 'SUPER JUNIOR', ...,
       'Charley Brownn', 'Avaleya and The GlitterHawks',
       'Renato pezzella| Renato Pezzella'], dtype=object)

Some of the songs contain more than one artist. Should be easy to clean them up

In [30]:
print(sorted(song_one['language'].unique()))

[-1.0, 3.0, 10.0, 17.0, 24.0, 31.0, 38.0, 45.0, 52.0, 59.0, nan]


Looks like language ids. There are only a handful of them. -1 or nan could be songs that have no lyrics

Taking a look at member

In [32]:
member.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')

In [38]:
print(member['city'].unique())
print(member['city'].nunique())

[ 1 13 12 15  5  6  4 11 18 22 14  8 20  3 17 10 16  9 21  7 19]
21


Only 21 cities. No NaN or empty cities. Seems like its mandatory to provide city when a user signs up.
<b><span style="color:red">No clue what cities these ids refer to</span></b>

In [41]:
bd = member['bd']
bd.describe()

count    34403.000000
mean        12.280935
std         18.170251
min        -43.000000
25%          0.000000
50%          0.000000
75%         25.000000
max       1051.000000
Name: bd, dtype: float64

In [45]:
bd.mode()

0    0
dtype: int64

In [50]:
bd.loc[bd.isna()]

Series([], Name: bd, dtype: int64)

In [55]:
bd_dropped_invalid_age = bd.drop(bd[bd <= 0].index)

In [60]:
print(bd_dropped_invalid_age.describe())
print(bd_dropped_invalid_age.mode())

count    14469.000000
mean        29.206027
std         17.041252
min          2.000000
25%         22.000000
50%         27.000000
75%         34.000000
max       1051.000000
Name: bd, dtype: float64
0    22
dtype: int64


~20,000 of 34404 users have age of 0. Since age is optional upon registration, 0 probably means user didnt provide their age. 
KKbox warned us there are outliers in the data. We can see age of -43, 1051 in the data. 

In [65]:
member['gender'].unique()

array([nan, 'female', 'male'], dtype=object)

In [68]:
member['gender'].value_counts()

male      7405
female    7096
Name: gender, dtype: int64

In [69]:
print(member['gender'].isna().sum(axis=0))

19902


More than half of the users didnt provide their gender. More males than females provided their gender. 

In [72]:
member['registered_via'].unique()

array([ 7,  4,  9,  3, 13, 16])

<b><span style="color:red">No clue what these means. </span></b>